In [2]:
!pip install google-generativeai requests beautifulsoup4 pandas

In [3]:
import google.generativeai as genai
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import re
from typing import Dict, List, Optional
import time

In [4]:
class CompetitorAnalysisConfig:
    """Configuration settings for the competitor analysis system"""

    def __init__(self):

        self.GEMINI_API_KEY = "ENTER_YOUR_KEY"
        self.GOOGLE_PLACES_API_KEY = "ENTER_YOUR_KEY"


        self.MODEL_NAME = "models/gemini-1.5-flash"


        self.SEARCH_RADIUS = 2000
        self.MAX_COMPETITORS = 10


        self.PEAK_HOURS_THRESHOLD = 70
        self.HIGH_FOOTFALL_THRESHOLD = 80

In [5]:
class CompetitorSearchTool:
    """Enhanced search tool for finding competitor information"""

    def __init__(self, config: CompetitorAnalysisConfig):
        self.config = config

    def search_competitors(self, location: str, business_type: str = "clothing store") -> List[Dict]:
        """
        Search for competitors in the specified location
        In a real implementation, this would integrate with Google Places API
        """

        mock_competitors = [
            {
                "name": "Fashion Central",
                "address": f"Near {location}",
                "type": "Contemporary Fashion",
                "distance": 0.3,
                "rating": 4.1,
                "price_level": 2,
                "business_hours": {
                    "monday": "10:00-22:00",
                    "tuesday": "10:00-22:00",
                    "wednesday": "10:00-22:00",
                    "thursday": "10:00-22:00",
                    "friday": "10:00-23:00",
                    "saturday": "09:00-23:00",
                    "sunday": "10:00-22:00"
                }
            },
            {
                "name": "Trendy Threads",
                "address": f"Central {location}",
                "type": "Youth Fashion",
                "distance": 0.5,
                "rating": 4.3,
                "price_level": 3,
                "business_hours": {
                    "monday": "11:00-21:00",
                    "tuesday": "11:00-21:00",
                    "wednesday": "11:00-21:00",
                    "thursday": "11:00-21:00",
                    "friday": "11:00-22:00",
                    "saturday": "10:00-22:00",
                    "sunday": "11:00-21:00"
                }
            },
            {
                "name": "Style Studio",
                "address": f"Main Street, {location}",
                "type": "Designer Wear",
                "distance": 0.8,
                "rating": 4.6,
                "price_level": 4,
                "business_hours": {
                    "monday": "12:00-20:00",
                    "tuesday": "12:00-20:00",
                    "wednesday": "12:00-20:00",
                    "thursday": "12:00-20:00",
                    "friday": "12:00-21:00",
                    "saturday": "11:00-21:00",
                    "sunday": "12:00-20:00"
                }
            }
        ]

        return mock_competitors

    def get_footfall_data(self, competitor_name: str) -> Dict:
        """
        Get footfall data for a specific competitor
        In real implementation, this would connect to footfall analytics APIs
        """
        import random

        current_hour = datetime.now().hour


        base_footfall = random.randint(40, 90)


        if 11 <= current_hour <= 13:
            base_footfall += random.randint(10, 20)
        elif 18 <= current_hour <= 20:
            base_footfall += random.randint(15, 25)
        elif current_hour < 10 or current_hour > 21:
            base_footfall -= random.randint(20, 30)

        base_footfall = max(10, min(100, base_footfall))

        return {
            "current_footfall": base_footfall,
            "avg_daily_visitors": random.randint(200, 500),
            "peak_hours": ["11:00-13:00", "18:00-20:00"],
            "busy_days": ["Friday", "Saturday", "Sunday"],
            "footfall_trend": "increasing" if base_footfall > 60 else "stable"
        }


In [6]:
class CompetitorAnalysisPipeline:
    """Main pipeline for competitor analysis using conversational AI"""

    def __init__(self, config: CompetitorAnalysisConfig):
        self.config = config
        self.search_tool = CompetitorSearchTool(config)


        genai.configure(api_key=config.GEMINI_API_KEY)
        self.model = genai.GenerativeModel(model_name=config.MODEL_NAME)


        self.conversation_history = []

    def search_and_analyze(self, location: str, user_query: str = "") -> Dict:
        """
        Main function that searches for competitors and generates analysis
        """
        print(f"🔍 Searching for competitors in {location}...")


        competitors = self.search_tool.search_competitors(location)


        detailed_competitors = []
        for comp in competitors:
            footfall_data = self.search_tool.get_footfall_data(comp["name"])
            comp.update(footfall_data)
            detailed_competitors.append(comp)


        analysis = self.generate_ai_analysis(location, detailed_competitors, user_query)

        return {
            "location": location,
            "competitors": detailed_competitors,
            "analysis": analysis,
            "timestamp": datetime.now().isoformat(),
            "total_competitors": len(detailed_competitors)
        }

    def generate_ai_analysis(self, location: str, competitors: List[Dict], user_query: str) -> str:
        """
        Generate comprehensive AI analysis using Gemini
        """

        competitor_context = self.format_competitor_data(competitors)

        prompt = f"""
        You are an expert retail market analyst specializing in clothing store competition analysis.

        LOCATION: {location}
        CURRENT TIME: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

        COMPETITOR DATA:
        {competitor_context}

        USER QUERY: {user_query if user_query else "Provide comprehensive competitor analysis"}

        Please provide a detailed analysis covering:

        1. **MARKET OVERVIEW**
           - Market saturation level
           - Competitive intensity
           - Price positioning landscape

        2. **FOOTFALL ANALYSIS**
           - Current footfall trends
           - Peak hours identification
           - Busiest competitors right now

        3. **COMPETITIVE POSITIONING**
           - Market gaps and opportunities
           - Competitor strengths and weaknesses
           - Differentiation opportunities

        4. **STRATEGIC RECOMMENDATIONS**
           - Immediate actions (next 24-48 hours)
           - Short-term strategies (1-4 weeks)
           - Long-term positioning (1-3 months)

        5. **TACTICAL INSIGHTS**
           - Optimal operating hours
           - Pricing strategy recommendations
           - Marketing timing suggestions

        Format your response in a professional, data-driven manner with specific actionable insights.
        Include relevant metrics and percentages where applicable.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating analysis: {str(e)}"

    def format_competitor_data(self, competitors: List[Dict]) -> str:
        """Format competitor data for AI analysis"""
        formatted_data = []

        for i, comp in enumerate(competitors, 1):
            formatted_data.append(f"""
            COMPETITOR {i}: {comp['name']}
            - Type: {comp['type']}
            - Distance: {comp['distance']}km
            - Rating: {comp['rating']}/5.0
            - Price Level: {'$' * comp['price_level']}
            - Current Footfall: {comp['current_footfall']}%
            - Daily Visitors: {comp['avg_daily_visitors']}
            - Peak Hours: {', '.join(comp['peak_hours'])}
            - Trend: {comp['footfall_trend']}
            """)

        return '\n'.join(formatted_data)

    def conversational_interface(self):
        """
        Interactive conversational interface for competitor analysis
        """
        print("🏪 Welcome to the Clothing Store Competitor Analysis System!")
        print("=" * 60)

        while True:
            print("\n🤖 How can I help you with competitor analysis today?")
            print("Examples:")
            print("- 'Analyze competitors in Koramangala, Bangalore'")
            print("- 'Find peak hours for nearby stores'")
            print("- 'What's the market saturation in my area?'")
            print("- Type 'quit' to exit")

            user_input = input("\n💬 Your query: ").strip()

            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("👋 Thanks for using the Competitor Analysis System!")
                break


            location = self.extract_location(user_input)
            if not location:
                location = input("📍 Please specify your location: ")


            print(f"\n🔄 Processing your request for {location}...")
            result = self.search_and_analyze(location, user_input)


            self.display_results(result)


            follow_up = input("\n❓ Any follow-up questions? (press Enter to continue or type your question): ")
            if follow_up.strip():
                print(f"\n🤖 Follow-up Analysis:")
                follow_up_analysis = self.generate_ai_analysis(location, result['competitors'], follow_up)
                print(follow_up_analysis)

    def extract_location(self, text: str) -> Optional[str]:
        """Extract location from user input using basic pattern matching"""

        location_patterns = [
            r'in\s+([A-Za-z\s,]+)',
            r'at\s+([A-Za-z\s,]+)',
            r'near\s+([A-Za-z\s,]+)',
            r'around\s+([A-Za-z\s,]+)'
        ]

        for pattern in location_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).strip()

        return None

    def display_results(self, result: Dict):
        """Display analysis results in a formatted manner"""
        print("\n" + "="*80)
        print(f"📊 COMPETITOR ANALYSIS REPORT - {result['location'].upper()}")
        print("="*80)

        print(f"\n📈 SUMMARY:")
        print(f"• Total Competitors Found: {result['total_competitors']}")
        print(f"• Analysis Timestamp: {result['timestamp']}")

        print(f"\n🏪 COMPETITORS OVERVIEW:")
        for i, comp in enumerate(result['competitors'], 1):
            status = "🔥 HIGH TRAFFIC" if comp['current_footfall'] > 70 else "📊 MODERATE" if comp['current_footfall'] > 40 else "📉 LOW"
            print(f"{i}. {comp['name']} ({comp['type']}) - {status}")
            print(f"   Distance: {comp['distance']}km | Rating: {comp['rating']}/5 | Footfall: {comp['current_footfall']}%")

        print(f"\n🤖 AI ANALYSIS:")
        print("-" * 50)
        print(result['analysis'])
        print("-" * 50)

    def generate_report_export(self, result: Dict) -> str:
        """Generate exportable report"""
        report = f"""
CLOTHING STORE COMPETITOR ANALYSIS REPORT
Generated: {result['timestamp']}
Location: {result['location']}

{result['analysis']}

DETAILED COMPETITOR DATA:
{self.format_competitor_data(result['competitors'])}
        """
        return report

In [ ]:
def main():
    """Main execution function"""

    config = CompetitorAnalysisConfig()


    pipeline = CompetitorAnalysisPipeline(config)


    pipeline.conversational_interface()


def quick_analysis(location: str):
    """Quick analysis function for immediate results"""
    config = CompetitorAnalysisConfig()
    pipeline = CompetitorAnalysisPipeline(config)

    print(f"🔍 Analyzing competitors in {location}...")
    result = pipeline.search_and_analyze(location)
    pipeline.display_results(result)

    return result


if __name__ == "__main__":
    print("🚀 Clothing Store Competitor Analysis Pipeline")
    print("Choose your preferred mode:")
    print("1. Interactive Conversational Mode")
    print("2. Quick Analysis Mode")

    choice = input("Enter your choice (1 or 2): ")

    if choice == "1":
        main()
    elif choice == "2":
        location = input("📍 Enter your location: ")
        quick_analysis(location)
    else:
        print("Invalid choice. Starting conversational mode...")
        main()


class AdvancedAnalytics:
    """Advanced analytics features for power users"""

    @staticmethod
    def generate_csv_report(result: Dict) -> str:
        """Generate CSV report of competitors"""
        df = pd.DataFrame(result['competitors'])
        filename = f"competitor_analysis_{result['location'].replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        df.to_csv(filename, index=False)
        return filename

    @staticmethod
    def trend_analysis(location: str, days: int = 7):
        """Analyze trends over multiple days (mock implementation)"""

        print(f"📈 Trend analysis for {location} over {days} days would be implemented here")
        return "Trend analysis feature coming soon!"


import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('competitor_analysis.log'),
        logging.StreamHandler()
    ]
)


"""
try:
    result = quick_analysis("Koramangala, Bangalore")
    print("✅ Analysis completed successfully!")
except Exception as e:
    logging.error(f"Analysis failed: {str(e)}")
    print("❌ Analysis failed. Please check the logs.")
"""

🚀 Clothing Store Competitor Analysis Pipeline
Choose your preferred mode:
1. Interactive Conversational Mode
2. Quick Analysis Mode
Enter your choice (1 or 2): 1
🏪 Welcome to the Clothing Store Competitor Analysis System!

🤖 How can I help you with competitor analysis today?
Examples:
- 'Analyze competitors in Koramangala, Bangalore'
- 'Find peak hours for nearby stores'
- 'What's the market saturation in my area?'
- Type 'quit' to exit

💬 Your query: Analyze competitors in Koramangala, Bangalore

🔄 Processing your request for Koramangala, Bangalore...
🔍 Searching for competitors in Koramangala, Bangalore...

📊 COMPETITOR ANALYSIS REPORT - KORAMANGALA, BANGALORE

📈 SUMMARY:
• Total Competitors Found: 3
• Analysis Timestamp: 2025-06-16T03:21:35.920690

🏪 COMPETITORS OVERVIEW:
1. Fashion Central (Contemporary Fashion) - 📊 MODERATE
   Distance: 0.3km | Rating: 4.1/5 | Footfall: 57%
2. Trendy Threads (Youth Fashion) - 📊 MODERATE
   Distance: 0.5km | Rating: 4.3/5 | Footfall: 62%
3. Style S